In [4]:

def filter_cf_data(country: str):
    """
    Get conflict forecast data for a specific country.
    
    Args:
        country (str): The name of the country to filter the data.
        window (int): The window size for the forecast.
        horizon (int): The horizon size for the forecast.
        
    Returns:
        pl.DataFrame: Filtered conflict forecast data.
    """
    
    # First, get the file listing
    API_URL = "http://api.backendless.com/C177D0DC-B3D5-818C-FF1E-1CC11BC69600/C5F2917E-C2F6-4F7D-9063-69555274134E/services/fileService/get-latest-file-listing"

    # Get the latest file listing
    resp = requests.get(API_URL)
    resp.raise_for_status()
    files = resp.json()

    # Find the specific file you want
    target_file = "conflictforecast_ons_armedconf_03.csv"
    file_url = None

    for file_info in files:
        if file_info["name"] == target_file:
            file_url = file_info["publicUrl"]
            break

    if file_url:
        
        # Download the CSV file
        csv_response = requests.get(file_url)
        csv_response.raise_for_status()
        
        # Read directly into Polars DataFrame
        cf_conflict = pl.read_csv(csv_response.content)
    else:
        print(f"File {target_file} not found in the listing")

    def iso_lookup(country):
        for c in pycountry.countries:
            if country.lower() in [c.name.lower(), getattr(c, 'official_name', '').lower(), getattr(c, 'alpha_3', '').lower()]:
                return c.alpha_3

        print(f"Country '{country}' not found in ISO lookup.")
        return None
    
    country = iso_lookup(country)
        
    return cf_conflict.filter(
        (pl.col("isocode") == country)
    )

country = "Sudan"  # ISO code for Sudan
    
cf_data = filter_cf_data(country)


In [41]:
# Convert period to datetime for better plotting
# Extract year and month from period (format YYYYMM)
df = cf_data.with_columns([
    pl.col("period").cast(str).str.slice(0, 4).alias("year"),
    pl.col("period").cast(str).str.slice(4, 6).alias("month")
])

# Create a date column by combining year and month
df = df.with_columns([
    pl.concat_str([pl.col("year"), pl.col("month")], separator="-").str.to_datetime("%Y-%m").alias("date")
])

# Filter for 2020 onwards
df_filtered = df.filter(pl.col("year").cast(int) >= 2020)

# Convert to pandas for easier plotting
pdf = df_filtered.select(["date", "ons_armedconf_03_all"]).to_pandas()

# Get the last datapoint for labeling
last_date = pdf["date"].iloc[-1]
last_value = pdf["ons_armedconf_03_all"].iloc[-1]

# Create the Plotly figure
fig = go.Figure()

# Add the gradient line plot using Scatter with marker colors
fig.add_trace(go.Scatter(
    x=pdf["date"], 
    y=pdf["ons_armedconf_03_all"],
    mode='lines+markers',
    line=dict(width=0),  # Hide the line, we'll use markers
    marker=dict(
        color=pdf["ons_armedconf_03_all"],  # Color based on values
        colorscale='RdYlBu_r',  # Red to Blue reversed (blue low, red high)
        size=3,
        line=dict(width=0),
        showscale=False,
        colorbar=dict(
            title="Conflict Probability"
        )
    ),
    name='Armed Conflict Forecast',
    connectgaps=True
))

# Create a smooth line with gradient effect by adding multiple segments
x_vals = pdf["date"]
y_vals = pdf["ons_armedconf_03_all"]

# Normalize values for color mapping
norm_vals = (y_vals - y_vals.min()) / (y_vals.max() - y_vals.min())

# Add line segments with different colors
for i in range(len(x_vals)-1):
    # Calculate color based on average of two consecutive points
    avg_val = (norm_vals.iloc[i] + norm_vals.iloc[i+1]) / 2
    
    # Convert normalized value to RGB color (blue to red)
    red = int(255 * avg_val)
    blue = int(255 * (1 - avg_val))
    color = f'rgb({red}, 0, {blue})'
    
    fig.add_trace(go.Scatter(
        x=[x_vals.iloc[i], x_vals.iloc[i+1]],
        y=[y_vals.iloc[i], y_vals.iloc[i+1]],
        mode='lines',
        line=dict(color=color, width=2),
        showlegend=False,
        hoverinfo='skip'
    ))

# Add a point and label for the last datapoint
fig.add_trace(go.Scatter(
    x=[last_date],
    y=[last_value],
    mode='markers+text',
    marker=dict(color='darkred', size=8),
    text=[f'{last_value:.3f}'],
    textposition='top right',
    textfont=dict(size=12, color='darkred'),
    showlegend=False,
    name='Latest Value'
))

# Update layout
fig.update_layout(
    title='Armed Conflict Probability Forecast',
    xaxis_title='Date',
    yaxis_title='Conflict Probability',
    font=dict(size=12, family='Arial(sans-serif)'),
    showlegend=False,
    width=900,
    height=500,
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Add grid and x-axis formatting
fig.update_xaxes(
    showgrid=True, 
    gridcolor='lightgray', 
    gridwidth=1,
    dtick="M3",  # Monthly ticks
    tickformat="%b",  # Show only month abbreviation for ticks
    tickangle=45,  # Rotate labels for better readability
    # Add year labels
    minor=dict(
        dtick="M12",  # Year intervals
        tickmode="linear"
    )
)

# Add custom year annotations
years = pdf["date"].dt.year.unique()
for year in sorted(years):
    # Find the first date of each year in the data
    year_data = pdf[pdf["date"].dt.year == year]
    if not year_data.empty:
        first_date = year_data["date"].min()
        fig.add_annotation(
            x=first_date,
            y=pdf["ons_armedconf_03_all"].min() - 0.05,  # Position below the plot
            text=str(year),
            showarrow=False,
            font=dict(size=12),
            xanchor="left"
        )

fig.update_yaxes(showgrid=True, gridcolor='lightgray', gridwidth=0.5)

# Show the plot
fig.show()

In [22]:
def plot_conflict_forecast(country: str):
    """
    Plot conflict probability forecast for a specific country with gradient colors.
    """
    import polars as pl
    import requests
    import pycountry
    import plotly.graph_objects as go
    import pandas as pd

    # Improved ISO3 code lookup
    def iso3(name):
        name = name.lower().strip()
        
        # Handle common country name mappings first
        country_mappings = {
            'united states': 'USA',
            'usa': 'USA',
            'us': 'USA',
            'america': 'USA',
            'united states of america': 'USA',
            'russia': 'RUS',
            'russian federation': 'RUS',
            'iran': 'IRN',
            'south korea': 'KOR',
            'north korea': 'PRK',
            'uk': 'GBR',
            'britain': 'GBR',
            'great britain': 'GBR',
            'united kingdom': 'GBR',
        }
        
        if name in country_mappings:
            return country_mappings[name]
        
        # Try exact matches first
        for c in pycountry.countries:
            names_to_check = [
                c.name.lower(),
                getattr(c, 'official_name', '').lower(),
                c.alpha_3.lower()
            ]
            if name in names_to_check:
                return c.alpha_3
        
        # Then try substring matching (but prioritize shorter matches)
        matches = []
        for c in pycountry.countries:
            names_to_check = [
                c.name.lower(),
                getattr(c, 'official_name', '').lower()
            ]
            for country_name in names_to_check:
                if country_name and (name in country_name or country_name in name):
                    matches.append((c.alpha_3, len(country_name), country_name))
        
        if matches:
            # Sort by length to prefer shorter, more precise matches
            matches.sort(key=lambda x: x[1])
            return matches[0][0]
        
        print(f"Country '{name}' not found in ISO lookup.")
        return None

    try:
        # Get latest file listing and find target file
        files = requests.get(
            "http://api.backendless.com/C177D0DC-B3D5-818C-FF1E-1CC11BC69600/C5F2917E-C2F6-4F7D-9063-69555274134E/services/fileService/get-latest-file-listing"
        ).json()
        file_url = next((f["publicUrl"] for f in files if f["name"] == "conflictforecast_ons_armedconf_03.csv"), None)
        if not file_url: raise ValueError("Target file not found.")

        # Read and filter data
        df = pl.read_csv(requests.get(file_url).content)
        
        iso = iso3(country)
        if not iso: raise ValueError(f"ISO code not found for {country}")
        
        df = df.filter(pl.col("isocode") == iso)
        if df.height == 0: raise ValueError(f"No data for {country}")

        # Parse dates and filter
        df = df.with_columns([
            pl.col("period").cast(str).str.slice(0, 4).alias("year"),
            pl.col("period").cast(str).str.slice(4, 6).alias("month"),
        ])
        df = df.with_columns([
            (pl.col("year") + "-" + pl.col("month")).str.to_datetime("%Y-%m").alias("date")
        ]).filter(pl.col("year").cast(int) >= 2020)
        if df.height == 0: raise ValueError(f"No data for {country} from 2020")

        pdf = df.select(["date", "ons_armedconf_03_all"]).to_pandas()
        x, y = pdf["date"], pdf["ons_armedconf_03_all"]
        norm = (y - y.min()) / (y.max() - y.min())

        fig = go.Figure()
        # Add colored line segments
        for i in range(len(x)-1):
            c = int(255 * ((norm.iloc[i] + norm.iloc[i+1])/2))
            fig.add_trace(go.Scatter(
                x=[x.iloc[i], x.iloc[i+1]], y=[y.iloc[i], y.iloc[i+1]],
                mode='lines', line=dict(color=f'rgb({c},0,{255-c})', width=2),
                showlegend=False, hoverinfo='skip'
            ))
        # Last value label
        fig.add_trace(go.Scatter(
            x=[x.iloc[-1]], y=[y.iloc[-1]], mode='markers+text',
            marker=dict(color='darkred', size=8),
            text=[f'{y.iloc[-1]:.2f}'], textposition='top right',
            textfont=dict(size=12, color='darkred'), showlegend=False
        ))

        fig.update_layout(
            title=f'Armed Conflict Probability Forecast - {country}',
            xaxis_title='Date', yaxis_title='Conflict Probability',
            font=dict(size=12, family='Arial, sans-serif'), showlegend=False,
            width=900, height=500, paper_bgcolor='white', plot_bgcolor='white'
        )
        fig.update_xaxes(
            showgrid=True, gridcolor='lightgray', gridwidth=1,
            dtick="M3", tickformat="%b", tickangle=45, automargin=True
        )
        fig.update_yaxes(showgrid=True, gridcolor='lightgray', gridwidth=0.5, automargin=True)
        
        # Add custom year annotations
        years = pdf["date"].dt.year.unique()
        for year in sorted(years):
            # Find the first date of each year in the data
            year_data = pdf[pdf["date"].dt.year == year]
            if not year_data.empty:
                first_date = year_data["date"].min()
                fig.add_annotation(
                    x=first_date,
                    y=pdf["ons_armedconf_03_all"].min()-(((pdf["ons_armedconf_03_all"].max())-(pdf["ons_armedconf_03_all"].min()))*0.1),  # Position below the plot
                    text=str(year),
                    showarrow=False,
                    font=dict(size=12),
                    xanchor="left"
                )
        
        return fig

    except Exception as e:
        print(f"Error in plot_conflict_forecast: {e}")
        import traceback
        traceback.print_exc()
        return None

# Example usage
fig = plot_conflict_forecast("Spain")
if fig: fig.show()